In [90]:
#importing the CSV file, original file can be found on GitHub page or at https://www.kaggle.com/datasets/sumitm004/forest-fire-area
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("forestfires.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       517 non-null    int64  
 1   Y       517 non-null    int64  
 2   month   517 non-null    object 
 3   day     517 non-null    object 
 4   FFMC    517 non-null    float64
 5   DMC     517 non-null    float64
 6   DC      517 non-null    float64
 7   ISI     517 non-null    float64
 8   temp    517 non-null    float64
 9   RH      517 non-null    int64  
 10  wind    517 non-null    float64
 11  rain    517 non-null    float64
 12  area    517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [91]:
from calendar import month_abbr
#addidng a new column where month is displayed as the Int8 monthNum
lower_ma = [m.lower() for m in month_abbr]

df['monthNum'] = df['month'].str.lower().map(lambda m: lower_ma.index(m)).astype('Int8')
df.head(10)

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,monthNum
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,3
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,10
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,10
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,3
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,3
5,8,6,aug,sun,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,0.0,8
6,8,6,aug,mon,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,0.0,8
7,8,6,aug,mon,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,0.0,8
8,8,6,sep,tue,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,0.0,9
9,7,5,sep,sat,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,0.0,9


In [92]:
#dropping the string values from the dataset as well as X and Y. X and Y were very specific to the Montesinho Natural Park and
#we want the model to be usable in any location
df = df.drop(columns=['X', 'Y','month','day'])
df.head(10)

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,monthNum
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,3
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,10
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,10
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,3
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,3
5,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,0.0,8
6,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,0.0,8
7,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,0.0,8
8,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,0.0,9
9,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,0.0,9


In [93]:
#converting area to area_int as this is both simple and will work for the model

df['area_int'] = df['area'].astype({"area": int})
#df = df.drop(columns=['area'])
df.head(10)

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,monthNum,area_int
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,3,0
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,10,0
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,10,0
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,3,0
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,3,0
5,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,0.0,8,0
6,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,0.0,8,0
7,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,0.0,8,0
8,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,0.0,9,0
9,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,0.0,9,0


Cross Validation

In [94]:
#Kfold CV

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import cross_val_score

model=DecisionTreeClassifier()
kfold_validation=KFold(10)

mod_score3=cross_val_score(model,X,y,cv=kfold_validation)
print(mod_score3)
#Overall accuracy of the model will be average of all values.
print(np.mean(mod_score3))

[0.37837838 0.30555556 0.16666667 0.33333333 0.22222222 0.27777778
 0.30555556 0.33333333 0.33333333 0.38888889]
0.3045045045045045


In [95]:
#Stratified kfold CV

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

sk_fold=StratifiedKFold(n_splits=10)

model=DecisionTreeClassifier()
mod_score4=cross_val_score(model,X,y,cv=sk_fold)
print(np.mean(mod_score4))
print(mod_score4)

0.28243243243243243
[0.32432432 0.19444444 0.25       0.22222222 0.13888889 0.27777778
 0.33333333 0.36111111 0.41666667 0.30555556]


c:\Users\krh52\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


In [96]:
#Repeated Random Test-Train Split

from sklearn.model_selection import ShuffleSplit

model=DecisionTreeClassifier()
s_split=ShuffleSplit(n_splits=10,test_size=0.30)
mod_score5=cross_val_score(model,X,y,cv=s_split)
print(mod_score5)
print(np.mean(mod_score5))

[0.30275229 0.29357798 0.24770642 0.22018349 0.32110092 0.26605505
 0.30275229 0.33027523 0.29357798 0.3853211 ]
0.29633027522935784


Using Scikit-learn SelectFromModel

In [97]:
#train test split

#split the dataset into train and test sets
from sklearn.model_selection import train_test_split

X, y = df[['FFMC','DMC','DC','ISI','temp','RH','wind','rain','monthNum']], df.area_int
  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) 

In [98]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel


sel = SelectFromModel(DecisionTreeClassifier(),threshold='median')
sel.fit(X_train,y_train)


selected_feat= X_train.columns[(sel.get_support())]
print(selected_feat)

Index(['FFMC', 'DMC', 'temp', 'RH', 'wind'], dtype='object')


In [99]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import precision_score, recall_score
  

X = X_train[['FFMC', 'DC', 'temp', 'RH', 'wind']]
y = y_train

X_test = X_test[['FFMC', 'DC', 'temp', 'RH', 'wind']]

tree_model = DecisionTreeClassifier()
tree_model.fit(X,y)

y_pred = tree_model.predict(X)
print('Results for decision tree on training data')
print('  Default settings')
print("Confusion Matrix")
print(confusion_matrix(y, y_pred))
print('Accuracy is  ', accuracy_score(y, y_pred))
print('Precision is ', precision_score(y, y_pred, average='weighted'))
print('Recall is    ', recall_score(y,y_pred, average='weighted'))
print('F1 is        ', f1_score(y, y_pred, average='weighted'))
print()

y_test_pred = tree_model.predict(X_test)
print('Results for decision tree on test data')
print('  Default settings')
print("Confusion Matrix")
print(confusion_matrix(y_test, y_test_pred))
print('Accuracy is  ', accuracy_score(y_test, y_test_pred))
print('Precision is ', precision_score(y_test, y_test_pred, average='weighted'))
print('Recall is    ', recall_score(y_test,y_test_pred, average='weighted'))
print('F1 is        ', f1_score(y_test, y_test_pred, average='weighted'))

Results for decision tree on training data
  Default settings
Confusion Matrix
[[191   0   0 ...   0   0   0]
 [  1  24   0 ...   0   0   0]
 [  1   0  18 ...   0   0   0]
 ...
 [  0   0   0 ...   1   0   0]
 [  0   0   0 ...   0   1   0]
 [  0   0   0 ...   0   0   1]]
Accuracy is   0.96398891966759
Precision is  0.9601080606675626
Recall is     0.96398891966759
F1 is         0.9598885305018607

Results for decision tree on test data
  Default settings
Confusion Matrix
[[47  2 10 ...  2  0  0]
 [ 4  1  0 ...  0  0  0]
 [ 4  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]]
Accuracy is   0.32051282051282054
Precision is  0.328917378917379
Recall is     0.32051282051282054
F1 is         0.3238087823108066


c:\Users\krh52\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\krh52\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\krh52\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
